# Download all pokemon images

This section will download all pokemon images into an "img" folder. 

In [147]:
from bs4 import BeautifulSoup
import requests
import os

def get_pokemon_image_url(pokedex_number, pokemon_name):
    # Construct the URL of the Pokemon page on the archives.bulbagarden.net
    url = f"https://archives.bulbagarden.net/wiki/File:{pokedex_number:04d}{pokemon_name}.png"
    
    # Send a GET request to the URL
    response = requests.get(url)
    
    # If the GET request is successful, proceed
    if response.status_code == 200:
        # Parse the page content with Beautiful Soup
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Find the img tag with the specific alt attribute
        img_tag = soup.find('img', alt=lambda x: x and x.startswith(f"File:{pokedex_number:04d}"))
        
        # If the img tag is found, extract and return the src attribute (image URL)
        if img_tag:
            return img_tag['src']
    return None


In [148]:
def download_pokemon_image(pokedex_number, pokemon_name, save_path, save_name):
    # Forming the URL
    url = get_pokemon_image_url(pokedex_number, pokemon_name)

    # Sending GET request to download the image
    response = requests.get(url)
    
    # If request was successful, save the image
    if response.status_code == 200:
        with open(os.path.join(save_path, f"{save_name}.png"), "wb") as file:
            file.write(response.content)
        print(f"Image for {pokemon_name} downloaded successfully!")
        return True 
    else:
        print(f"Error downloading image for {pokemon_name}.")
        return False

In [149]:
def adjust_pokemon_name_simple(name):
    
    name_l = name.lower()
    # Handle special cases
    if name_l in ["nidoran-m", "nidoran-f"]:
        return "Nidoran"
    
    # Capitalize each word if hyphenated
    return '-'.join([word.capitalize() for word in name.split('-')])

In [143]:
import pypokedex

failed_pokemon = []

for dex in range(1 , 3):  # Range of pokemon that you wash to retrieve
    try:
        p = pypokedex.get(dex=dex)
        pokemon_name = adjust_pokemon_name_simple(p.name) # Capitalize the name and handle exceptions
        download_pokemon_image(dex, pokemon_name, "img", p.name)
    except Exception as e:
        print(f"An error occurred while processing Pokémon number {dex}: {e}")
        failed_pokemon.append(pokemon_name)


Image for Bulbasaur downloaded successfully!
Image for Ivysaur downloaded successfully!


Some pokemon images failed to download, need more robust searching to capture those

In [144]:
# Now we have most of the images, but we're missing the pokemon from failed_pokemon
# This will be a bit more robust in the searching for them

# more robust adjustment that tries multiple possibilities
def adjust_pokemon_name(name):
    base_name = name.split('-')[0]
    
    # Handle special cases
    special_cases = {
        "Farfetchd": "Farfetch%27d",
        "Mr-Mime": "Mr._Mime",
        "Mr-Rime": "Mr._Rime",
        "Mime-Jr": "Mime_Jr.",
        "Nidoran-M": "Nidoran",
        "Nidoran-F": "Nidoran",
        "Type-Null": "Type_Null",
        "Sirfetchd": "Sirfetch%27d",
        "Flabebe": "Flabébé",
        "Sinistcha": "HOME1013"
    }
    
    if name in special_cases:
        return [special_cases[name]]
    
    # General cases
    possibilities = [base_name.capitalize()]
    
    # If there's a hyphen, add versions with and without the hyphen
    if '-' in name:
        # All hyphens and each word capitalized
        possibilities.append('-'.join([word.capitalize() for word in name.split('-')]))

        # All hyphens with only the first word capitalized
        words = name.split('-')
        possibilities.append(words[0].capitalize() + '-' + '-'.join([word.lower() for word in words[1:]]))

        # Hyphen replaced with underscore
        possibilities.append('_'.join([word.capitalize() for word in name.split('-')]))

    
    return possibilities



In [145]:
# Loop through all failed pokemon and try again to get their image
still_failed = []

for pokemon_name in failed_pokemon:
    try:
        dex = pypokedex.get(name=pokemon_name).dex
        adjusted_names = adjust_pokemon_name(pokemon_name)
        
        success = False  # Flag to check if the image download was successful

        # Try all of the possible names to match the URL of the image
        for adjusted_name in adjusted_names:
            try:
                download_pokemon_image(dex, adjusted_name, "img", pokemon_name)
                success = True
                break
            except Exception as e:
                print(f"Failed to download image for {adjusted_name} using dex number {dex}: {e}")

        if not success:
            still_failed.append(dex)
    
    except Exception as e:
        print(f"Error processing {pokemon_name}")



# Get all pokemon

In [150]:
import json
import os
import pypokedex

pokemon_data = []

for dex in range(1, 1018):
    try:
        # get pokemon
        p = pypokedex.get(dex=dex)
        
        # print status
        print(f"{p.name.capitalize()}")

        # get moves
        first_key = list(p.moves.keys())[0]
        move_data = p.moves[first_key]
        moves = list(set([move.name for move in move_data]))

        base_stats = {
            'hp': p.base_stats.hp,
            'attack': p.base_stats.attack,
            'defense': p.base_stats.defense,
            'sp_atk': p.base_stats.sp_atk,
            'sp_def': p.base_stats.sp_def,
            'speed': p.base_stats.speed
        }
        data = {
            'name': p.name.capitalize(),
            'health': p.base_stats.hp,  # Assume a default health of 100
            'types': p.types,
            'image_path': f'img\\{p.name.capitalize()}.png',
            'moves': moves,  # Could truncate this to limit moves?
            'base_stats': base_stats
        }
        pokemon_data.append(data)
    except Exception as e:
        print(f"An error occurred while processing Pokémon number {dex}: {e}")

# Check if file already exists to prevent accidental overwriting
filename = 'pokemon_data.json'
if os.path.exists(filename):
    overwrite = input(f"{filename} already exists. Do you want to overwrite it? (y/n): ")
    if overwrite.lower() != 'y':
        print("Operation cancelled.")
        exit()

# Write to pokemon_data.json
with open(filename, 'w') as file:
    json.dump(pokemon_data, file, indent=4)


Bulbasaur
Ivysaur
Venusaur
Charmander
Charmeleon
Charizard
Squirtle
Wartortle
Blastoise
Caterpie
Metapod
Butterfree
Weedle
Kakuna
Beedrill
Pidgey
Pidgeotto
Pidgeot
Rattata
Raticate
Spearow
Fearow
Ekans
Arbok
Pikachu
Raichu
Sandshrew
Sandslash
Nidoran-f
Nidorina
Nidoqueen
Nidoran-m
Nidorino
Nidoking
Clefairy
Clefable
Vulpix
Ninetales
Jigglypuff
Wigglytuff
Zubat
Golbat
Oddish
Gloom
Vileplume
Paras
Parasect
Venonat
Venomoth
Diglett
Dugtrio
Meowth
Persian
Psyduck
Golduck
Mankey
Primeape
Growlithe
Arcanine
Poliwag
Poliwhirl
Poliwrath
Abra
Kadabra
Alakazam
Machop
Machoke
Machamp
Bellsprout
Weepinbell
Victreebel
Tentacool
Tentacruel
Geodude
Graveler
Golem
Ponyta
Rapidash
Slowpoke
Slowbro
Magnemite
Magneton
Farfetchd
Doduo
Dodrio
Seel
Dewgong
Grimer
Muk
Shellder
Cloyster
Gastly
Haunter
Gengar
Onix
Drowzee
Hypno
Krabby
Kingler
Voltorb
Electrode
Exeggcute
Exeggutor
Cubone
Marowak
Hitmonlee
Hitmonchan
Lickitung
Koffing
Weezing
Rhyhorn
Rhydon
Chansey
Tangela
Kangaskhan
Horsea
Seadra
Goldeen
Seakin

In [25]:
p5.types

['fire']

# Find pokemon moves

In [4]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

url = 'https://bulbapedia.bulbagarden.net/wiki/List_of_moves'

response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

#tables = pd.read_html(url)

# Now `tables` is a list of dataframes, each one representing a table from the webpage
# You can access individual tables like this:
#move_table = tables[0]


In [10]:
table_html = soup.find('table')

In [33]:
df = pd.read_html(str(table_html))[0]

df = df.drop(0).reset_index(drop=True)

df.columns = df.iloc[0]

df = df.drop(0).reset_index(drop=True)

df



C:\Users\ytsop\AppData\Local\Temp\ipykernel_10788\3650037193.py:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table_html))[0]


,#,Name,Type,Category,PP,Power,Accuracy,Gen
0,1,Pound,Normal,Physical,35,40,100%,I
1,2,Karate Chop,Fighting,Physical,25,50,100%,I
2,3,Double Slap,Normal,Physical,10,15,85%,I
3,4,Comet Punch,Normal,Physical,15,18,85%,I
4,5,Mega Punch,Normal,Physical,20,80,85%,I
...,...,...,...,...,...,...,...,...
904,???,Electro Shot,Electric,Special,NaN,NaN,NaN,IX
905,???,Thunderclap,Electric,Special,NaN,NaN,NaN,IX
906,???,Tachyon Cutter,Steel,Special,NaN,NaN,NaN,IX
907,???,Upper Hand,Fighting,Physical,NaN,NaN,NaN,IX


In [34]:
# Define a function to check if a value can be converted to an integer
def is_integer(value):
    try:
        int(value)
        return True
    except (ValueError, TypeError):
        return False

# Filter the DataFrame to keep only rows where 'Power' can be converted to an integer
df = df[df['Power'].apply(is_integer)]

# Convert the 'Power' column to integers, scale to fit with the HP
df['Power'] = df['Power'].astype(int) * .2

# Convert 'Name' of moves to match version in pokedex
# Converting the 'name' column to lowercase and replacing spaces with hyphens
df['Name'] = df['Name'].str.lower().str.replace(' ', '-')


df.head(20)

C:\Users\ytsop\AppData\Local\Temp\ipykernel_10788\2760911222.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Power'] = df['Power'].astype(int) * .2
C:\Users\ytsop\AppData\Local\Temp\ipykernel_10788\2760911222.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Name'] = df['Name'].str.lower().str.replace(' ', '-')


,#,Name,Type,Category,PP,Power,Accuracy,Gen
0,1,pound,Normal,Physical,35,8.0,100%,I
1,2,karate-chop,Fighting,Physical,25,10.0,100%,I
2,3,double-slap,Normal,Physical,10,3.0,85%,I
3,4,comet-punch,Normal,Physical,15,3.6,85%,I
4,5,mega-punch,Normal,Physical,20,16.0,85%,I
5,6,pay-day,Normal,Physical,20,8.0,100%,I
6,7,fire-punch,Fire,Physical,15,15.0,100%,I
7,8,ice-punch,Ice,Physical,15,15.0,100%,I
8,9,thunder-punch,Electric,Physical,15,15.0,100%,I
9,10,scratch,Normal,Physical,35,8.0,100%,I


In [35]:
# Type advantage dictionary

type_advantages = {
    "Normal": {
        "strength_against": [],
        "weakness_against": ["Rock", "Steel"]
    },
    "Fire": {
        "strength_against": ["Grass", "Ice", "Bug", "Steel"],
        "weakness_against": ["Fire", "Water", "Rock", "Dragon"]
    },
    "Water": {
        "strength_against": ["Fire", "Ground", "Rock"],
        "weakness_against": ["Water", "Electric", "Grass", "Dragon"]
    },
    "Electric": {
        "strength_against": ["Water", "Flying"],
        "weakness_against": ["Electric", "Ground", "Grass", "Dragon"]
    },
    "Grass": {
        "strength_against": ["Water", "Ground", "Rock"],
        "weakness_against": ["Fire", "Grass", "Poison", "Flying", "Bug", "Dragon", "Steel"]
    },
    "Ice": {
        "strength_against": ["Grass", "Ground", "Flying", "Dragon"],
        "weakness_against": ["Fire", "Water", "Ice", "Steel"]
    },
    "Fighting": {
        "strength_against": ["Normal", "Ice", "Rock", "Dark", "Steel"],
        "weakness_against": ["Poison", "Flying", "Psychic", "Bug", "Fairy"]
    },
    "Poison": {
        "strength_against": ["Grass", "Fairy"],
        "weakness_against": ["Poison", "Ground", "Rock", "Ghost"]
    },
    "Ground": {
        "strength_against": ["Fire", "Electric", "Poison", "Rock", "Steel"],
        "weakness_against": ["Grass", "Ice", "Water", "Bug"]
    },
    "Flying": {
        "strength_against": ["Grass", "Fighting", "Bug"],
        "weakness_against": ["Electric", "Rock", "Steel"]
    },
    "Psychic": {
        "strength_against": ["Fighting", "Poison"],
        "weakness_against": ["Psychic", "Steel"]
    },
    "Bug": {
        "strength_against": ["Grass", "Psychic", "Dark"],
        "weakness_against": ["Fire", "Fighting", "Poison", "Flying", "Ghost", "Steel", "Fairy"]
    },
    "Rock": {
        "strength_against": ["Fire", "Ice", "Flying", "Bug"],
        "weakness_against": ["Fighting", "Ground", "Steel"]
    },
    "Ghost": {
        "strength_against": ["Psychic", "Ghost"],
        "weakness_against": ["Dark"]
    },
    "Dragon": {
        "strength_against": ["Dragon"],
        "weakness_against": ["Steel"]
    },
    "Dark": {
        "strength_against": ["Psychic", "Ghost"],
        "weakness_against": ["Fighting", "Dark", "Fairy"]
    },
    "Steel": {
        "strength_against": ["Ice", "Rock", "Fairy"],
        "weakness_against": ["Fire", "Water", "Electric", "Steel"]
    },
    "Fairy": {
        "strength_against": ["Fighting", "Dragon", "Dark"],
        "weakness_against": ["Fire", "Poison", "Steel"]
    }
}


In [36]:
import json


# list to hold the data
data = []

# iterate over the rows of the DataFrame
for index, row in df.iterrows():
    name = row[1].lower()  # assuming the name is in the second column
    power = row[5]  # assuming the power is in the sixth column
    type_ = row[2]  # assuming the type is in the third column
    
    # get the strengths and weaknesses from the mapping
    strength_against = type_advantages.get(type_, {}).get("strength_against", None)
    weakness_against = type_advantages.get(type_, {}).get("weakness_against", None)
    
    # create a dictionary for this row
    move_data = {
        "name": name,
        "power": power,
        "strength_against": strength_against,
        "weakness_against": weakness_against
    }
    
    # append the dictionary to the list
    data.append(move_data)

# convert the list to JSON
json_data = json.dumps(data, indent=4)

# write the JSON data to a file
with open('moves_data.json', 'w') as f:
    f.write(json_data)


C:\Users\ytsop\AppData\Local\Temp\ipykernel_10788\3033047564.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  name = row[1].lower()  # assuming the name is in the second column
C:\Users\ytsop\AppData\Local\Temp\ipykernel_10788\3033047564.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  power = row[5]  # assuming the power is in the sixth column
C:\Users\ytsop\AppData\Local\Temp\ipykernel_10788\3033047564.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by 

In [31]:
df.sort_values(by="Power", ascending=False)

,#,Name,Type,Category,PP,Power,Accuracy,Gen
152,153,Explosion,Normal,Physical,5,50.0,100%,I
700,701,Pulverizing Pancake,Normal,Physical,1,42.0,—%,VII
657,658,Catastropika,Electric,Physical,1,42.0,—%,VII
119,120,Self-Destruct,Normal,Physical,5,40.0,100%,I
724,725,Menacing Moonraze Maelstrom,Ghost,Special,1,40.0,—%,VII
...,...,...,...,...,...,...,...,...
34,35,Wrap,Normal,Physical,20,3.0,90%,I
39,40,Poison Sting,Poison,Physical,35,3.0,100%,I
30,31,Fury Attack,Normal,Physical,20,3.0,85%,I
131,132,Constrict,Normal,Physical,35,2.0,100%,I
